In [2]:
import sklearn as sk
import pandas as pd
from scipy.stats import rv_discrete
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
#from skopt import BayesSearchCV #intento fallido con esta libreria 'scikit-optimize'
from scipy.stats import uniform
from sklearn.metrics import accuracy_score

## Preprocesamiento

In [8]:
equipos = pd.read_csv('../Data/equipos_data.csv')
equipos = equipos[['equipo','posicion','valor_mercado','aforo']]
historico = pd.read_csv('../Data/historico_data.csv')
historico = historico[['equipo1','equipo2','mes','fin_de_semana','publico']]

##### Ver si se llenan los estadios en algunos partidos 

In [32]:
se_llena = pd.merge(historico,equipos, left_on='equipo1', right_on='equipo')
se_llena[se_llena['publico'] == se_llena['aforo']]

Se llena el estadio en 157 partidos

### Obtener variables dummies para entrenar los modelos

In [9]:
historico = pd.get_dummies(historico, prefix=['equipo1','equipo2'])

## Modelos Predictivos

In [10]:
X = historico.drop('publico',axis=1)
y = historico['publico']

In [11]:
X = X.to_numpy()

In [12]:
y = y.to_numpy()

In [13]:
X

array([[ 8,  1,  0, ...,  0,  0,  0],
       [ 4,  0,  0, ...,  0,  0,  0],
       [ 6,  1,  0, ...,  0,  0,  0],
       ...,
       [11,  1,  0, ...,  0,  0,  0],
       [ 3,  0,  0, ...,  0,  0,  0],
       [11,  1,  0, ...,  0,  0,  0]], dtype=int64)

### Regresion Lineal

In [14]:
from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
clf = LinearRegression(normalize=True)
clf.fit(X,y)
scores = clf.predict(X)
mean_squared_error(y,scores)**(1/2)

4919.510646449128

### Random Forest

In [15]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(max_depth = 100,random_state=0)
forest.fit(X, y)
scores = forest.predict(X)
mean_squared_error(y,scores)**(1/2)

2488.684547167605

In [6]:
print(rv_discrete(a=0,b=10000, inc= 150))

In [ ]:
# creamos stratified k-fold splits
kf = KFold(n_splits=5, shuffle=True, random_state=3)
best_estimator=[] # lista que guardara los mejores estimadores
print('ok')
for train, test in kf.split(X):
    print(train,test)
    X_train, X_test, y_train, y_test = X[train],X[test], y[train], y[test]

    for a in range(10): # PARA CADA SPLIT REPETIMOS 10 VECES
        # estimamos los mejores parámetros en base al score f1 (según documentacion mas adecuado para clases binarias)
        # como es nuestro caso, esteriotipado o no esteriotipado
        regresor = MLPRegressor() 
        distributions = dict(hidden_layer_sizes=[0,150,300,450,600,750,900,1050,1200,1350,1500],
                            activation=['identity', 'logistic', 'tanh', 'relu'],
                            solver=['lbfgs', 'sgd', 'adam'],
                            learning_rate=['constant', 'invscaling', 'adaptive'])
    #     print('ok')

    #     # prueba realizada con gridSearch para ver orden de magnitud de C, sin embargo, pese a salir siempre como optimo 10,
    #     # al realizar la validación cruzada en las siguientes celdas, hemos encontrado que valores entre 0 y 4 dan mejores resultados

        # estimamos mejores parámetros y ajustamos (gracias a GridSearch, ocurre similar con RandomizedSearch)
        clf = RandomizedSearchCV(regresor, distributions, scoring='accuracy', verbose=3)
        print('ok')
        clf.fit(X_train, y_train)
        print(clf.best_params_)

        # añadimos el mejor estimador de cada iteración para posteriormente realizar validación cruzada con el 
        # resto de mejores estimadores
        best_estimator.append(clf.best_estimator_)

        # predecimos con los mejores parámetros (por defecto)
        y_pred = clf.predict(X_test) 

        # calculamos f1 score (no será por lo que nos guiemos en último lugar)
        print('Accuracy:', accuracy_score(y_test, y_pred, average='weighted'))

In [18]:
from sklearn.neighbors import KNeighborsRegressor

# creamos k-fold splits
kf = KFold(n_splits=5, shuffle=True, random_state=3)
best_estimator=[] # lista que guardara los mejores estimadores

for train, test in kf.split(X):
    X_train, X_test, y_train, y_test = X[train],X[test], y[train], y[test]

    for a in range(10): # PARA CADA SPLIT REPETIMOS 10 VECES
        # estimamos los mejores parámetros en base al score f1 (según documentacion mas adecuado para clases binarias)
        # como es nuestro caso, esteriotipado o no esteriotipado
        regresor = KNeighborsRegressor() 
        distributions = dict(n_neighbors=[3,5,10,15,20,25,30,35,40,45,50],
                            weights=['uniform', 'distance'],
                            leaf_size=[2,5,10,15,20,25,30,35,40,45,50])

    #     # prueba realizada con gridSearch para ver orden de magnitud de C, sin embargo, pese a salir siempre como optimo 10,
    #     # al realizar la validación cruzada en las siguientes celdas, hemos encontrado que valores entre 0 y 4 dan mejores resultados

        # estimamos mejores parámetros y ajustamos (gracias a GridSearch, ocurre similar con RandomizedSearch)
        clf = RandomizedSearchCV(regresor, distributions, scoring='neg_mean_squared_error', cv=5)
        clf.fit(X_train, y_train)
        print(clf.best_params_)

        # añadimos el mejor estimador de cada iteración para posteriormente realizar validación cruzada con el 
        # resto de mejores estimadores
        best_estimator.append(clf.best_estimator_)

        # predecimos con los mejores parámetros (por defecto)
        y_pred = clf.predict(X_test) 

        # calculamos f1 score (no será por lo que nos guiemos en último lugar)
    #     print('Accuracy:', accuracy_score(y_test, y_pred, average='weighted'))

ok
[   0    1    2 ... 3797 3798 3799] [   6   21   27   30   34   38   57   58   59   69   70   72   76   87
   91  108  111  115  120  122  140  154  157  164  168  170  173  175
  188  192  194  197  202  210  211  215  216  222  226  228  231  236
  244  246  250  261  263  265  266  269  272  273  285  287  293  302
  317  318  321  322  323  325  328  330  332  339  346  349  356  370
  371  373  377  388  389  398  403  404  407  414  422  424  437  439
  445  447  448  454  458  464  469  471  481  490  493  499  509  511
  517  528  529  532  533  536  546  549  557  562  577  581  583  591
  593  597  602  617  623  626  649  654  658  660  668  671  691  692
  693  696  697  701  710  711  718  719  725  731  741  749  768  785
  800  805  808  811  818  820  821  824  828  838  841  849  869  870
  871  877  879  880  883  901  911  914  915  916  922  933  934  940
  942  958  967  971  973  976  977  989  990 1011 1013 1026 1032 1036
 1037 1041 1042 1046 1047 1048 1052 10

{'weights': 'uniform', 'n_neighbors': 30, 'leaf_size': 30}
[   0    2    3 ... 3797 3798 3799] [   1    7    8   14   15   19   24   39   45   49   51   53   54   55
   56   60   63   64   67   68   74   75   89   93   95   98  102  107
  109  118  127  130  132  135  136  138  142  143  145  152  155  158
  159  171  174  177  178  187  189  190  207  214  218  225  230  237
  254  262  268  286  294  309  313  315  316  320  327  333  336  338
  342  348  351  352  353  354  358  376  380  385  386  415  417  423
  428  429  430  431  442  444  449  452  457  461  462  465  477  479
  491  502  506  516  523  526  539  543  547  548  555  560  564  570
  574  585  587  594  595  598  608  616  618  624  638  639  644  648
  650  653  665  674  677  678  682  694  698  707  712  716  717  720
  721  723  726  727  736  742  746  747  757  761  762  766  771  774
  777  778  780  783  784  786  787  801  806  809  813  816  817  819
  831  839  842  844  847  854  855  861  872  886  8